In [80]:
from requests_html import HTMLSession
import numpy as np
import re
import newspaper

In [87]:
session = HTMLSession()

def extract_site_urls(root_url):
    r = session.get(root_url)
    site_urls = []
    site_name_words = []
    for l in r.html.links:
        match = re.match( r'http:\/\/mediabiasfactcheck\.com\/(.+)\/', l)
        if match:
            site_urls.append(l) 
            words = match.groups(1)[0].split('-')
            site_name_words.append(words)
            
    return site_urls, site_name_words

In [88]:
def get_cite_root_for_url(url, cite_words):
    r = session.get(url)
    links = []
    for l in r.html.links:
        if cite_words[0] in l and 'mediabiasfactcheck' not in l and 'google' not in l and len(l) < 40:
            links.append(l)
    if len(links) == 0:
        return None
    lengths = [len(l) for l in links]
    min_id = np.argmin(lengths)
    return links[min_id]

In [89]:
def get_site_roots(urls, words):
    site_roots = []
    for url, site_words in zip(urls, words):
        root = get_cite_root_for_url(url, site_words)
        if root:
            site_roots.append(root)
            print('Found site root {}'.format(root))
    return site_roots

In [90]:
def extract_article_urls(site_urls):
    article_urls = []
    for site_url in site_urls: 
        paper = newspaper.build(site_url)
        urls = [a.url for a in paper.articles if '-' in a.url]
        print('Extracted {} articles from site {}'.format(len(urls) ,site_url))
        article_urls += urls 
    return article_urls

In [91]:
def download_article(url):
    article = Article(url)
    article.download()
    article.parse()
    article.nlp()
    print('Downloaded article: {}'.format(article.title))
    return {
        'title': article.title,
        'authors': article.authors,
        'date': article.publish_date,
        'text': article.text,
        'keywords': article.keywords,
        'summary': article.summary
    }

def scrape_articles(links, save_name, ):
    articles = []
    for url in links:
        a = download_article(url)
        articles.append(a)
        
    with open(save_name, 'wb') as f:
        dill.dump(articles, f)
    print('Articles save into "articles.pkl"')

In [92]:
urls, words = extract_site_urls('https://mediabiasfactcheck.com/left/')
site_roots = get_site_roots(urls, words)
article_urls = extract_article_urls(site_roots)
scrape_articles(article_urls, 'articles_left')

Found site root http://modernliberals.com/
Found site root http://earthfirstjournal.org/
Found site root http://www.huffingtonpost.com/
Found site root http://www.takepart.com/
Found site root https://www.inequalitymedia.org/
Found site root http://rabble.ca/
Found site root http://news.groopspeak.com/
Found site root http://elisabethparker.com/
Found site root http://www.drudge.com/
Found site root http://www.lavendermagazine.com/
Found site root http://mondoweiss.net/
Found site root http://www.salon.com/
Found site root http://jezebel.com/
Found site root https://this.org/
Found site root http://economyincrisis.org/
Found site root http://mediamatters.org/
Found site root http://gizmodo.com/
Found site root http://www.msmagazine.com/
Found site root http://www.dailydot.com/
Found site root https://bitchmedia.org/
Found site root http://politicalupdater.com/
Found site root http://leftoverrights.com/
Found site root http://usuncut.com/
Found site root https://www.liberationnews.org/


Extracted 0 articles from site http://washingtonmonthly.com/
Extracted 0 articles from site http://elitedaily.com/
Extracted 0 articles from site http://www.tribunemagazine.org/
Extracted 0 articles from site https://thewalrus.ca/
Extracted 0 articles from site http://the-daily.buzz/
Extracted 0 articles from site https://rewire.news/
Extracted 0 articles from site http://americannewsx.com/
Extracted 0 articles from site https://www.pressprogress.ca/
Extracted 0 articles from site http://www.redpepper.org.uk/
Extracted 0 articles from site http://newscorpse.com/
Extracted 0 articles from site https://crimethinc.com/
Extracted 6 articles from site https://www.jacobinmag.com/
Extracted 0 articles from site http://revolution-news.com/
Extracted 0 articles from site http://www.bluedotdaily.com/
Extracted 0 articles from site http://www.msnbc.com/
Extracted 0 articles from site http://www.politicususa.com/
Extracted 0 articles from site http://www.vanityfair.com/
Extracted 5 articles from s

NameError: name 'Article' is not defined